# Etapa do Fine-tuning do Transformers
Etapa do fine-tuning no modelo DistilBERT pré-treinado nos nossos dados, para depois, salvarmos o modelo e seus respectivos pesos treinados.

Nesta etapa realizamos:
* Definição do modelo;
* Fine-tuning do modelo;
* Avaliação do modelo;
* Salvamento do modelo e dos pesos treinados.

## Table of Contents
* [Pacotes](#1)
* [Carregando os Dados](#2)
* [Transformers](#3)
* [Transfer Learning](#4)
    * [Fine-tuning](#4.1)

<a name="1"></a>
## Pacotes
Pacotes que foram utilizados no sistema:
* [transformers](https://huggingface.co/docs/transformers/index): fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração;
* [datasets](https://huggingface.co/docs/datasets/index): é uma biblioteca para acessar e compartilhar facilmente datasets para tarefas de áudio, visão computacional e processamento de linguagem natural (NLP);
* [scikit-learn](https://scikit-learn.org/stable/): biblioteca open-source de machine learning;
* [src](../src/): pacote com todos os códigos de todas as funções utilitárias criadas para esse sistema. Localizado dentro do diretório `../src/`.

In [49]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

import os
import sys
PROJECT_ROOT = os.path.abspath( # Obtendo a versão absoluta normalizada do path raíz do projeto
    os.path.join( # Concatenando os paths
        os.getcwd(), # Obtendo o path do diretório dos notebooks
        os.pardir # Obtendo a string constante usada pelo OS para fazer referência ao diretório pai
    )
)
# Adicionando o path à lista de strings que especifica o path de pesquisa para os módulos
sys.path.append(PROJECT_ROOT)
from src.transformers_finetuning import *

> **Nota**: os códigos para as funções utilitárias utilizadas nesse sistema estão no script `transformers_finetuning.py` dentro do diretório `../src/`.

<a name="2"></a>
## Carregando os Dados
Vamos ler cada um dos subsets dentro de seus respectivos diretórios dentro do diretório `../data/preprocessed/` e plotar cada um deles.

In [34]:
train_set = Dataset.load_from_disk('../data/preprocessed/train_dataset')
val_set = Dataset.load_from_disk('../data/preprocessed/validation_dataset')
test_set = Dataset.load_from_disk('../data/preprocessed/test_dataset')
print(f'Shape do train set: {train_set}\nShape do validation set: {val_set}\nShape do test set: {test_set}')

Shape do train set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10156
})
Shape do validation set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3385
})
Shape do test set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3386
})


<a name="3"></a>
## Transformers
<img align='center' src='../figures/transformers.png' style='width:400px;'>
Transformer é um modelo puramente baseado em attention que foi desenvolvido pelo Google para solucionar alguns problemas com RNNs, como o de ser difícil de explorar totalmente as vantagens da computação paralela, devido aos seus problemas decorrentes de sua estrutura sequencial. Em um RNN seq2seq, precisamos passar por cada palavra de nosso input, de forma sequencial pelo encoder, e é feito de uma forma sequencial similar pelo decoder, sem computação paralela. Por esse motivo, não há muito espaço para cálculos paralelos. Quanto mais palavras temos na sequência de input, mais tempo será necessário para processá-la.

Com sequências grandes, ou seja, com muitos $T$ steps sequenciais, as informações tendem a se perder na rede (loss of information) e problemas de vanishing gradients surgem relacionados ao comprimento de nossas input sequences. LSTMs e GRUs ajudam um pouco com esses problemas, mas mesmo essas arquiteturas param de funcionar bem quando tentam processar sequências muito longas devido ao `information bottleneck`.
* `Loss of information`: é mais difícil saber se o sujeito é singular ou plural à medida que nos afastamos do sujeito.
* `Vanishing gradients`: quando calculamos o backprop, os gradients podem se tornar muito pequenos e, como resultado, o modelo não aprenderá nada.

Os transformers se baseiam em attention e não exigem nenhum cálculo sequencial por layer, sendo necessário apenas um único step. Além disso, os steps de gradient que precisam ser realizados do último output para o primeiro output em um transformers são apenas 1. Para RNNs, o número de steps aumenta com sequências mais longas. Por fim, os transformers não sofrem com problemas de vanishing gradients relacionados ao comprimento das sequências.

Transformers não usam RNNs, attention é tudo o que precisamos, e apenas algumas transformações lineares e não lineares geralmente são incluídas. O modelo transformers foi introduzido em 2017 por pesquisadores do Google, e desde então, a arquitetura do transformer se tornou padrão para LLMs. Os transformers revolucionaram o campo de NLP.

O modelo transformers usa `scaled dot-product attention`. A primeira forma da attention é muito eficiente em termos de computação e memória, porque consiste apenas em operações de multiplicações de matrizes. Esse mecanismo é o núcleo do modelo e permite que o transformers cresça e se torne mais complexo, sendo mais rápido e usando menos memória do que outras arquiteturas de modelos comparáveis.

No modelo transformers, usaremos a `Multi-Head Attention layer`, essa layer é executada em paralelo e tem vários mecanismos scaled dot-product attention $h$ e várias transformações lineares dos input queries $Q$, keys $K$ e values $V$. Nessa layer, as transformações lineares são parâmetros treináveis.
$$\text{ Attention}(Q, K, V) = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} \right) V$$
<img align='center' src='../figures/attention.png' style='width:600px;'>

Os transformers `encoders` começam com um módulo multi-head attention que executa a `self-attention` na input sequence. Isso é seguido por uma residual connection e normalização, em seguida, uma feed forward layer e outra residual connection e normalização. A encoder layer é repetida $N_x$ vezes.
* Self-attention: cada palavra no input corresponde a cada outra palavra no input sequence.
* Graças à self-attention layer, o encoder fornecerá uma representação contextual de cada um de nossos inputs.


O `decoder` é construído de forma similar ao encoder, com módulos multi-head attention, residual connections e normalização. O primeiro módulo de attention é mascarado (`Masked Self-Attention`) de forma que cada posição atenda apenas às posições anteriores, ele bloqueia o fluxo de informações para a esquerda. O segundo módulo de attention (`Encoder-Decoder Attention`) pega o output do encoder e permite que o decoder atenda a todos os itens. Toda essa decoder layer também é repetida várias $N_x$ vezes, uma após a outra.
$$\text{ Masked Self-Attention } = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} + M \right) = \mathrm{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} + \text{ mask matrix } \begin{pmatrix} 0 & -\infty & -\infty \\ 0 & 0 & -\infty \\ 0 & 0 & 0 \end{pmatrix} \right)$$

Os transformers também incorporam um `positional encoding stage` ($PE$), que codifica a posição de cada input na sequência, ou seja, as informações sequenciais. Isso é necessário porque os transformers não usam RNNs, mas a ordem das palavras é relevante para qualquer idioma. A positional encoding é treinável, assim como as word embeddings.
$$\begin{align*}
& \text{PE}_{(\text{pos, }2i)} = \text{sin}\left( \frac{\text{pos}}{10000^{\frac{2i}{d}}} \right) \\
& \text{PE}_{(\text{pos, }2i + 1)} = \text{cos}\left( \frac{\text{pos}}{10000^{\frac{2i}{d}}} \right)
\end{align*}$$

Primeiro, é calculado o embedding sobre o input e as positional encodings são aplicadas. Então, isso vai para o encoder que consiste em várias layers de módulos de Multi-Head Attention, em seguida, o decoder recebe a sequência de output deslocada um step para a direita e os outputs do encoder. O output do decoder é transformado em probabilidades de outputs usando uma layer linear com uma ativação softmax. Essa arquitetura é fácil de paralelizar em comparação com os modelos RNNs e pode ser treinada com muito mais eficiência em várias GPUs. Ela também pode ser escalada para aprender várias tarefas em datasets cada vez maiores. Transformers são uma ótima alternativa aos RNNs e ajudam a superar esses problemas em NLP e em muitos campos que processam dados sequenciais.

Faremos o fine-tuning no modelo DistilBERT, que é um modelo Transformer pequeno, rápido, barato e leve, treinado pela destilação do modelo base BERT. Ele tem 40% menos parâmetros que o bert-base-uncased, roda 60% mais rápido e preserva mais de 95% do desempenho do BERT conforme medido no benchmark GLUE (General Language Understanding Evaluation).

[Hugging Face](https://huggingface.co/) (🤗) é o melhor recurso para transformers pré-treinados. Suas bibliotecas de código aberto simplificam o download, o fine-tuning e o uso de modelos de transformers como DeepSeek, BERT, Llama, T5, Qwen, GPT-2 e muito mais. E a melhor parte, podemos usá-los junto com TensorFlow, PyTorch ou Flax. Neste notebook, utilizo transformers 🤗 para usar o modelo `DistilBERT` para classificação de sentimento. Para a etapa do pré-processamento, usamos o tokenizador (no notebook `03_preprocessing.ipynb`), e no fine-tuning do checkpoint do DistilBERT `distilbert-base-uncased-finetuned-sst-2-english` pré-treinado no código abaixo. Para isso inicializamos a classe DistilBertForSequenceClassification e definidos o modelo pré-treinado desejado.

<a name="4"></a>
## Transfer Learning
Para o modelo transformers, utilizamos a técnica que transfer learning. Ela é uma das ideias mais poderosas do deep learning, que às vezes, podemos pegar o conhecimento que a rede neural aprendeu em uma tarefa e aplicar esse conhecimento em uma tarefa separada. Há 3 vantagens principais no transfer learning são:
* Reduz o tempo de treinamento.
* Melhora as previsões.
* Nos permite usar datasets menores.

Se estamos criando um modelo, em vez de treinarmos os pesos do 0, a partir de uma inicialização aleatória, geralmente progredimos muito mais rápido baixando pesos que outra pessoa já treinou por dias/semanas/meses na arquitetura da rede neural, as usamos como pré-treinamento, e as transferimos para uma nova tarefa semelhante na qual possamos estar interessados. Isso significa que muitas vezes podemos baixar pesos de algoritmos open-source, que outras pessoas levaram semanas ou meses para descobrir, e então, usamos isso como uma inicialização muito boa para nossa rede neural.

<a name="4.1"></a>
### Fine-tuning
Com o modelo pré-treinado, aplicamos o fine-tuning. Então, pegamos os pesos de um modelo pré-treinado existente, utilizando transfer learning e, em seguida, ajustamos um pouco para garantir que funcionem na tarefa específica em que estamos trabalhando. Digamos que pré-treinamos um modelos que prevê a avaliação de filmes, e agora vamos criar um modelo para avaliar cursos. Uma maneira de fazer isso é, bloqueando todos os pesos que já temos pré-treinados e, em seguida, adicionamos uma nova output layer, ou talvez, uma nova feed forward layer e uma output layer que serão treinadas, enquanto mantemos o restante bloqueado e, em seguida, treinamos apenas a nossa nova rede, as novas layers que acabamos de adicionar. Podemos descongelar lentamente as layers, uma de cada vez.

Muitas das features de baixo nível que o modelo pré-treinado aprendeu a partir de um corpus muito grande, como a estrutura do texto, a natureza do texto, isso pode ajudar nosso algoritmo a se sair melhor na tarefa de classificação de sentimentos e mais rápido ou com menos dados, porque talvez o modelo tenha aprendido o suficiente como são as estruturas de textos diferentes e parte desse conhecimento será útil. Após excluirmos a output layer de um modelo pré-treinado, não precisamos necessariamente criar apenas a output layer, mas podemos criar várias novas layers.

Precisamos remover a output layer do modelo pré-treinado e adicionar a nossa, porque a rede neural pode ter uma softmax output layer que gera um dos 1000 labels possíveis. Então removemos essa output layer e criamos a nossa própria output layer, nesse caso uma ativação sigmoid.

* Com um training set pequeno, pensamos no restante das layers como `congeladas`, então congelamos os parâmetros dessas layers, e treinamos apenas os parâmetros associados à nossa output layer. Dessa forma obteremos um desempenho muito bom, mesmo com um training set pequeno.

* Com um training set maior, nós podemos congelar menos layers e então treinar as layers que não foram congeladas e a nossa nova output layer. Podemos usar as layers que não estão congeladas como inicialização e usar o gradient descent a partir delas, ou também podemos eliminar essas layers que não estão congeladas e usamos nossas próprias hidden layers novas e nossa própria output layer. Qualquer um desses métodos pode valer a pena tentar.

* Com um training set muito maior usamos essa rede neural pré-treinada e os seus pesos como inicialização e treinamos toda a rede neural, apenas alterando a output layer, com labels que nos importamos.

Definindo o checkpoint do modelo pré-treinado que faremos o fine-tuning.

In [12]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

Definindo os hiperparâmetros com o objeto `TrainingArguments`, usando o objeto `Trainer` do Hugging Face para realizar o fine-tuning do modelo.
* O modelo pré-treinado com o fine-tuning já está sendo salvo no diretório `../models/`, definido na definição dos hiperparâmetros.

In [14]:
# Hiperparâmetros do fine-tuning
training_args = TrainingArguments(
    output_dir='../models/transformers_results',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=20,
    weight_decay=1e-1,
    eval_strategy='steps',
    lr_scheduler_type='reduce_lr_on_plateau',
    logging_steps=100
)
# Objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=f1_metric
)
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
100,0.690200,0.592717,0.684184
200,0.592300,0.547271,0.697679
300,0.523900,0.585440,0.712016
400,0.511200,0.488176,0.756529
500,0.486700,0.502221,0.761378
600,0.521900,0.474515,0.775508
700,0.509900,0.474097,0.764560
800,0.478600,0.448737,0.778758
900,0.475400,0.487454,0.767549
1000,0.501600,0.445698,0.818367


TrainOutput(global_step=2540, training_loss=0.4370420598608302, metrics={'train_runtime': 25970.5016, 'train_samples_per_second': 0.782, 'train_steps_per_second': 0.098, 'total_flos': 2690677801500672.0, 'train_loss': 0.4370420598608302, 'epoch': 2.0})

Avaliando o desempenho do modelo com fine-tuning no train e validation set.

In [44]:
print(f'Avaliação do train set: {trainer.evaluate(train_set)["eval_f1_score"]:.4f}\nAvaliação do validation set: {trainer.evaluate(val_set)["eval_f1_score"]:.4f}')

Avaliação do train set: 0.9178
Avaliação do validation set: 0.8523


Avaliando o desempenho do modelo final com fine-tuning no test set.

In [47]:
print(f'Avaliação do test set: {trainer.evaluate(test_set)["eval_f1_score"]:.4f}')

Avaliação do test set: 0.8518
